In [2]:
import pandas as pd
import sklearn as skr
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from dateutil.relativedelta import relativedelta
import os
os.chdir('D:/Dataset/PRD')
import sys
sys.path.append('D:/gitln/WorkCode/BaseFunc/')
import purchase.PurchaseBase as pbase
total_balance = pd.read_csv('base.csv')
total_balance = pbase.add_timestamp(total_balance,time_index='date')
import purchase.PurchaseFeatures as pfeature

In [4]:
total_balance = pfeature.extract_is_feature(total_balance)
labels = ['total_purchase_amt','total_redeem_amt']
date_indexs = ['week','year','month','weekday','day']
total_balance = pfeature.encode_data(total_balance)
feature = total_balance[
    [x for x in total_balance.columns if x not in date_indexs]]
purchase_feature_seems_useless = [
      #样本量太少，建模时无效；但若确定这是一个有用规则，可以对结果做修正
      'is_work_on_sunday',
      #中位数差异不明显
      'is_first_week']
temp = np.abs(feature[[x for x in feature.columns 
                       if x not in ['total_redeem_amt', 'date'] ]
                     ].corr('spearman')['total_purchase_amt'])
purchase_feature_low_correlation = list(set(temp[temp < 0.1].index))

d:\softwares\python\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [5]:
total_balance = pfeature.extract_distance_feature(total_balance)
purchase_feature_seems_useless += [
                                  'dis_to_holiday',
                                  'dis_from_startofmonth',
                                  'dis_from_middleofmonth']
feature = total_balance[
    [x for x in total_balance.columns if x not in date_indexs]]
dis_feature_indexs = [
    x for x in feature.columns if (
        x not in date_indexs + labels + ['date']) & ('dis' in x)]
def dis_change(x):
    if x > 5:
        x = 10
    return x
dis_holiday_feature = [
    x for x in total_balance.columns if 'dis' in x and 'holi' in x]
dis_month_feature = [
    x for x in total_balance.columns if 'dis' in x and 'month' in x]
total_balance[dis_holiday_feature] = total_balance[
    dis_holiday_feature].applymap(dis_change)
total_balance[dis_month_feature] = total_balance[
    dis_month_feature].applymap(dis_change)
temp = np.abs(feature[[x for x in feature.columns 
                       if ('dis' in x) | (x in ['total_purchase_amt']) 
                      ]].corr()['total_purchase_amt'])
purchase_feature_low_correlation += list(set(temp[temp < 0.1].index) )

In [6]:
total_balance = pfeature.extract_peak_feature(total_balance)
feature = total_balance[
    [x for x in total_balance.columns if x not in date_indexs]]
temp = np.abs(feature[
    [x for x in feature.columns if ('peak' in x) or (
        'valley' in x) or (x in ['total_purchase_amt']) ]
].corr()['total_purchase_amt'])
temp

total_purchase_amt          1.000000
dis_from_purchase_peak      0.007173
dis_from_purchase_valley    0.006313
Name: total_purchase_amt, dtype: float64

In [7]:
# 生成周期因子并合并到数据集

weekday_rate_list = []
day_rate_list = []
for i in range(3, 10):
    weekday_rate, day_rate = pfeature.generate_rate(total_balance, i,labels)
    weekday_rate_list.append(weekday_rate.reset_index(drop=True))
    day_rate_list.append(day_rate.reset_index(drop=True))

weekday_rate_list = pd.concat(weekday_rate_list).reset_index(drop=True)
day_rate_list = pd.concat(day_rate_list).reset_index(drop=True)
total_balance = pd.merge(total_balance, weekday_rate_list,
                         on=['weekday','month'], how='left')
total_balance = pd.merge(total_balance, day_rate_list, 
                         on=['day','month'], how='left')

# 对周期因子进行特殊处理

for i in [x for x in total_balance.columns 
          if 'rate' in x and x not in labels + date_indexs]:
    total_balance[i] = total_balance[i].fillna(
        np.nanmedian(total_balance[i]))
    
# 剔除相关性低的特征

feature = total_balance.drop(date_indexs, axis=1)

D:/gitln/WorkCode/BaseFunc\purchase\PurchaseFeatures.py:281: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  (pure_balance['date'] >= datetime.date(2014,3,1)
D:/gitln/WorkCode/BaseFunc\purchase\PurchaseFeatures.py:282: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  ) & (pure_balance['date'] < datetime.date(2014, month_index, 1))]


In [10]:
total_balance = pfeature.get_amtfeature_with_time(total_balance, 
                                                  labels, date_indexs)
# 对动态特征做特殊处理

for i in [x for x in total_balance.columns 
          if '_weekday_' in x and x not in labels + date_indexs]:
    total_balance[i] = total_balance[i].fillna(np.nanmedian(total_balance[i]))

D:/gitln/WorkCode/BaseFunc\purchase\PurchaseFeatures.py:322: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  df_tmp_ = df_tmp_[(df_tmp_['date']>=datetime.date(2014,3,3))]
d:\softwares\python\lib\site-packages\numpy\lib\nanfunctions.py:1112: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [11]:
# 剔除对数据集划分不明显的特征

purchase_feature_seems_useless += ['is_gonna_work_tomorrow',
                                   'is_fourth_week',
                                   'weekday_onehot_4']

In [12]:
import purchase.PurchaseHelper as phelper

In [14]:
l = phelper.mvtest()

name_list = []
Tn_list = []
p_list = []
for i in [i for i in purchase_feature_low_correlation
          if 'is' in i or 'discret' in i]:
    pair = l.test(feature['total_purchase_amt'], feature[i])
    name_list.append(str(i))
    Tn_list.append(pair['Tn'])
    p_list.append(pair['p-value'][0])
temp = pd.DataFrame([name_list,Tn_list]).T.sort_values(1)
temp[1] = np.abs(temp[1])
feature_saved_from_mv_purchase = list(temp.sort_values(1, ascending=False)[temp[1] > 0.5984][0])


D:/gitln/WorkCode/BaseFunc\purchase\PurchaseHelper.py:24: RuntimeWarning: invalid value encountered in less_equal
  return len(x[(x <= s) & (y == t)]) / len(y[y == t])
D:/gitln/WorkCode/BaseFunc\purchase\PurchaseHelper.py:21: RuntimeWarning: invalid value encountered in less_equal
  return len(x[x <= s]) / self._n
d:\softwares\python\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  del sys.path[0]


In [16]:
feature = feature[[x for x in feature.columns 
                   if (
    x not in purchase_feature_low_correlation + purchase_feature_seems_useless) or\
                   (x in feature_saved_from_mv_purchase )]]

purchase_cors = feature.corr()
purchase_cors['total_purchase_amt'] = np.abs(
    purchase_cors['total_purchase_amt'])
feature_lists = list(purchase_cors.sort_values(by='total_purchase_amt',
                                               ascending=False).index)[2:]
feature_temp = feature.dropna()

In [17]:
# 这里要注意 保留的时候按照相关性降序排序 剔除按照相关性升序排序的顺序
thershold = 0.8
for i in range(len(feature_lists)):
    for k in range(len(feature_lists)-1, -1, -1):
        if i >= len(feature_lists) or k >= len(feature_lists) or i == k:
            break
        if np.abs(np.corrcoef(feature_temp[feature_lists[i]], feature_temp[feature_lists[k]])[0][1]) > thershold:
            higher_feature_temp = feature_temp[feature_lists[i]] * feature_temp[feature_lists[k]]
            if np.abs(np.corrcoef(feature_temp[feature_lists[i]], higher_feature_temp)[0][1]) <= thershold:
                name = str(feature_lists[i]) + '%%%%' + str(feature_lists[k])
                feature_temp[name] = higher_feature_temp
                feature[name] = feature[feature_lists[i]] * feature[feature_lists[k]]
                feature_lists.append(name)
            feature_temp = feature_temp.drop(feature_lists[k], axis=1)
            feature_lists.remove(feature_lists[k])

d:\softwares\python\lib\site-packages\numpy\lib\function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
d:\softwares\python\lib\site-packages\numpy\lib\function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
d:\softwares\python\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [18]:
feature = feature[[
    x for x in feature_lists if x not in labels] + labels + ['date']]

In [ ]:
feature.to_csv('purchase_feature_droped_0614.csv',index=False)

['total_purchase_predicted_by_cycle',
 'dis_to_holiendday',
 'dis_from_holiday',
 'is_firstday_of_holiday',
 'dis_to_work',
 'total_redeem_amt',
 'is_lastday_of_holiday',
 'redeem_dayrate',
 'dis_from_work',
 'is_tailmonth',
 'dis_from_nowork',
 'redeem_weekdayrate',
 'is_premonth',
 'dis_to_nowork',
 'dis_from_holiendday',
 'weekday_onehot_5',
 'weekday_onehot_1',
 'purchase_weekdayrate',
 'dis_to_nowork%%%%dis_from_purchase_peak']

In [ ]:
['dis_from_holiday',
 'is_holiday',
 'dis_to_work',
 'dis_from_work',
 'dis_from_holiendday',
 'dis_from_nowork',
 'is_firstday_of_holiday',
 'is_tailmonth',
 'is_premonth',
 'is_lastday_of_holiday',
 'is_third_week',
 'is_work',
 'dis_to_nowork',
 'redeem_weekdayrate',
 'redeem_dayrate',
 'dis_from_purchase_peak',
 'purchase_dayrate',
 'purchase_weekdayrate',
 'is_holiday%%%%dis_to_holiendday']